In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
#from langchain.schema import Document

from langchain_core.documents import Document

#from langchain_community.vectorstores import chroma
from langchain_chroma import Chroma
#utility imports
import numpy as np
from typing import List




d:\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sample_docs=[
"""
1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.
Connectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).
Key Contacts: 7022588703 / 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, 
offering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.
 For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B.Tech students and 
 average packages around 15 LPA for top branches. The Corporate and Industry Relations (CIR) cell is particularly active here, 
 facilitating recruitment with global giants like Amazon, SAP Labs, and Walmart Global Tech. While smaller in size compared to Coimbatore, 
 the campus provides modern laboratories, design tools, and clean, hygiene-focused hostels that are highly rated by students


""",

"""
2. Coimbatore Campus (Main Campus) 
Location: Nestled in Ettimadai, this is the largest campus, housing over 12,000 students and 1,500 faculty members.
Academic Range: Offers 120+ programs across UG, PG, and Doctoral levels, including Engineering, Business, and Social Work.
Research: Home to an Internal Quality Assurance Cell (IQAC) and numerous specialized research centers.
Key Contacts: 0422-2685509 / 0422-2685510.

The Coimbatore campus, established in 1994, serves as the administrative headquarters of the university and is a sprawling 400-acre "oasis" nestled 
in the Western Ghats at Ettimadai. As of 2026, it supports over 12,000 students and nearly 1,500 faculty members across 120+ undergraduate,
 postgraduate, and doctoral programs. The campus is renowned for its eco-friendly initiatives, including massive forestation that 
 transformed once-barren land into a lush environment home to exotic birds like peacocks. Infrastructure highlights include a built-up 
 area of 28 lakh square feet, eight hostels accommodating 5,000 students, and an Olympic-standard swimming pool (50m x 25m). 
 Academic life is centered around the Schools of Engineering and Business, alongside Departments of Social Work and Communication. 
 For the 2026 placement season, the campus maintains a strong record with top recruiters like Google, Microsoft, and Intel, reporting 
 a highest package of 60 LPA and an average of 9 LPA.
""",

"""
3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for blending science with spirituality, offering programs in Computing, Engineering, Biotechnology, and Ayurveda.
Key Dates (2026): Even semester classes for 2026 are scheduled to commence on January 19, 2026.
Key Contacts: 7034024264 / 0476-2805280.

Situated in the serene coastal village of Vallikavu, Kerala, the Amritapuri campus spans 80 acres and is a unique blend of
 modern science and spiritual values. It features over one million square feet of built-up area, housing six prestigious schools 
 and more than 20 specialized research centers. The campus is highly residential, with approximately 95 percent of students living in hostels, 
 many of which are located within the ashram premises. Academic programs cover diverse fields including Biotechnology, Ayurveda, Engineering,
   and Social Sciences. Amritapuri is a research powerhouse, hosting the Humanitarian Technology (HuT) Labs, which recently launched the IOT-based 
   "PRABHA" robot and is hosting the 2026 IEEE International Conference on Robotics and Mechatronics. Facilities for students include high-speed 
   computer labs with over 600 systems, extensive sports complexes for cricket and football, and strictly vegetarian dining that promotes a holistic
     lifestyle.

"""



]

In [9]:
import os

data_dir = "data1"
os.makedirs(data_dir, exist_ok=True)

for i, doc in enumerate(sample_docs):
    with open(f"{data_dir}/doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(doc)

print("Documents saved in:", data_dir)


Documents saved in: data1


In [10]:
#STEP1:  -----> DOCUMENT LOADING
from langchain_community.document_loaders import DirectoryLoader,TextLoader

# Load documents from directory
loader = DirectoryLoader(
    "data1", 
    glob="*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview:")
print(documents[0].page_content[:200] + "...")


Loaded 3 documents

First document preview:

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tec...


In [11]:
documents

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="\n1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\n For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B

In [12]:
#step2----->DOCUMENT SPLITTING
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,
    separators=[" "]  # Hierarchy of separators
)
chunks=text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 11 chunks from 3 documents

Chunk example:
Content: 1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academi...
Metadata: {'source': 'data1\\doc_0.txt'}


In [13]:
chunks

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content='1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre'),
 Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="/ 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\

In [14]:

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name="amrita_clg"
)

vectorstore.persist()

print("Vectors stored:", vectorstore._collection.count())


Vectors stored: 11


C:\Users\DELL\AppData\Local\Temp\ipykernel_14036\1931119460.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
#just to load--without duplicating the vectors

persist_directory = "./chroma_db"

# Load existing vectorstore
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=OpenAIEmbeddings(),
    collection_name="amrita_clg"
)

print(f"Vector store has {vectorstore._collection.count()} vectors")


Vector store has 11 vectors


In [15]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model_name="gpt-5-nano"
)


In [18]:
## Convert vector store to retriever
retriever=vectorstore.as_retriever(
    search_kwargs={"k":3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F370201310>, search_kwargs={'k': 3})

In [19]:
##CREATING RAG CHAIN USING LCEL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [20]:
# Create a custom prompt
from langchain_core.prompts import ChatPromptTemplate
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question 
about the college and place. Donot hallucinate and do not guess anything.
be crystal clear to the asked question.
If you don't know the answer based on the context, say you don't know.
give the specific details and support.

Context:
{context}

Question: {query}

Answer:""")
custom_prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])

In [21]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F370201310>, search_kwargs={'k': 3})

In [22]:
#output doc for the promts...(combining all the chunks from the retriever for the prompt.)
def combi(docs):
    return "\n".join(doc.page_content for doc in docs)


In [24]:
rag_chain_lcel=(
    {
        "context":retriever|combi,
        "query":RunnablePassthrough()
    }
    |custom_prompt
    |llm
    |StrOutputParser()
)
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F370201310>, search_kwargs={'k': 3})
           | RunnableLambda(combi),
  query: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': Fal

In [26]:
response=rag_chain_lcel.invoke("what is the location on amritapuri campus in 1000 words")
response

'The location of the Amritapuri Campus, as described in the provided context, is Vallikavu, a serene coastal village in the Indian state of Kerala. The campus is situated along the coastline, giving it a distinctive seaside setting that combines natural beauty with an academic environment. In terms of size, the Amritapuri Campus spans about 80 acres, which translates into a substantial footprint on the coastside where education and spiritual values are integrated.\n\nKey geographic and contextual details about the location include:\n- City and state: Vallikavu village, Kerala, India.\n- Setting: Coastal, with the campus positioned along the shoreline. The coastal location is repeatedly emphasized, underscoring its importance to the campus identity.\n- Campus size: Approximately 80 acres of land, indicating a large, semi-rural campus with room for extensive facilities.\n- Built environment: The campus has more than one million square feet of built-up area, reflecting a substantial physi

In [27]:
#to test query_rag_lcel()

In [28]:
from IPython.display import display, Markdown

display(Markdown(response))


The location of the Amritapuri Campus, as described in the provided context, is Vallikavu, a serene coastal village in the Indian state of Kerala. The campus is situated along the coastline, giving it a distinctive seaside setting that combines natural beauty with an academic environment. In terms of size, the Amritapuri Campus spans about 80 acres, which translates into a substantial footprint on the coastside where education and spiritual values are integrated.

Key geographic and contextual details about the location include:
- City and state: Vallikavu village, Kerala, India.
- Setting: Coastal, with the campus positioned along the shoreline. The coastal location is repeatedly emphasized, underscoring its importance to the campus identity.
- Campus size: Approximately 80 acres of land, indicating a large, semi-rural campus with room for extensive facilities.
- Built environment: The campus has more than one million square feet of built-up area, reflecting a substantial physical infrastructure to support its academic programs and research activities.
- Community and landscape: The description highlights a serene environment typical of a coastal village in Kerala, suggesting a calm, inviting campus atmosphere that blends modern education with a spiritual orientation.

Within this location, the Amritapuri Campus is described as a unique blend of modern science and spiritual values. This positioning is not only about the academic programs but also about the physical and cultural setting that the location fosters. The coastal setting contributes to the campus’s ambiance, offering a backdrop that supports both rigorous scientific work and reflective, value-based learning. The fact that the campus sits in a rural coastal village rather than in a dense urban center reinforces its distinctive character and the experiential opportunities it provides to students and researchers.

The campus layout and facilities, as described in the context, further illuminate how the location supports its mission:
- Academic and research facilities: The campus houses six prestigious schools and more than 20 specialized research centers. While these details describe the campus’s intellectual footprint, they are contingent on the large 80-acre site, which provides the space necessary for a multi-institutional academic ecosystem to operate within the same geographic area.
- Residential nature: The Amritapuri Campus is described as highly residential, with about 95 percent of students living in hostels. Many of these hostels are located within the ashram premises, illustrating how the location supports a close-knit, campus-life-centric environment. The proximity of student housing to academic and research facilities is a practical outcome of the campus’s coastal, village setting and sizable land area.
- Campus identity: The combination of “modern science and spiritual values” is both a curricular identity and a reflection of the location’s cultural milieu. Although not a geographic attribute alone, this identity is reinforced by the environment in which the campus sits—Vallikavu’s coastal surroundings and Kerala’s broader heritage of spirituality and learning.

In sum, the location of the Amritapuri Campus can be characterized as follows:
- It is physically located in Vallikavu, Kerala, on the coast.
- It occupies a sizable 80-acre site along the coastline, providing ample space for a large built environment and extensive facilities.
- The campus’s built-up area exceeds one million square feet, indicating substantial infrastructure within a coastal setting.
- The location supports a residential campus life, with the majority of students housed in hostels, many within the ashram precincts, fostering a cohesive, campus-centered community.
- The coastal location complements the campus’s mission to blend science with spiritual values, situating academic pursuits within a serene, natural environment that is conducive to both rigorous research and reflective learning.

If you are seeking a precise description of “where” the Amritapuri Campus sits within Vallikavu, Kerala, the essential takeaway is that it lies along the coastline in the serene village of Vallikavu, occupying about 80 acres and featuring a large, integrated campus with multiple schools, research centers, and residential facilities. The emphasis on coastline and village setting is a defining element of its location, shaping the daily life, atmosphere, and educational experience of students, faculty, and researchers who operate there.